# Information Retrieval 1#
## Assignment 2: Retrieval models [100 points] ##

In this assignment you will get familiar with basic and advanced information retrieval concepts. You will implement different information retrieval ranking models and evaluate their performance.

We provide you with a Indri index. To query the index, you'll use a Python package ([pyndri](https://github.com/cvangysel/pyndri)) that allows easy access to the underlying document statistics.

For evaluation you'll use the [TREC Eval](https://github.com/usnistgov/trec_eval) utility, provided by the National Institute of Standards and Technology of the United States. TREC Eval is the de facto standard way to compute Information Retrieval measures and is frequently referenced in scientific papers.

This is a **groups-of-three assignment**, the deadline is **Wednesday, January 31st**. Code quality, informative comments and convincing analysis of the results will be considered when grading. Submission should be done through blackboard, questions can be asked on the course [Piazza](piazza.com/university_of_amsterdam/spring2018/52041inr6y/home).

### Technicalities (must-read!) ###

The assignment directory is organized as follows:
   * `./assignment.ipynb` (this file): the description of the assignment.
   * `./index/`: the index we prepared for you.
   * `./ap_88_90/`: directory with ground-truth and evaluation sets:
      * `qrel_test`: test query relevance collection (**test set**).
      * `qrel_validation`: validation query relevance collection (**validation set**).
      * `topics_title`: semicolon-separated file with query identifiers and terms.

You will need the following software packages (tested with Python 3.5 inside [Anaconda](https://conda.io/docs/user-guide/install/index.html)):
   * Python 3.5 and Jupyter
   * Indri + Pyndri (Follow the installation instructions [here](https://github.com/nickvosk/pyndri/blob/master/README.md))
   * gensim [link](https://radimrehurek.com/gensim/install.html)
   * TREC Eval [link](https://github.com/usnistgov/trec_eval)

### TREC Eval primer ###
The TREC Eval utility can be downloaded and compiled as follows:

    git clone https://github.com/usnistgov/trec_eval.git
    cd trec_eval
    make

TREC Eval computes evaluation scores given two files: ground-truth information regarding relevant documents, named *query relevance* or *qrel*, and a ranking of documents for a set of queries, referred to as a *run*. The *qrel* will be supplied by us and should not be changed. For every retrieval model (or combinations thereof) you will generate a run of the top-1000 documents for every query. The format of the *run* file is as follows:

    $query_identifier Q0 $document_identifier $rank_of_document_for_query $query_document_similarity $run_identifier
    
where
   * `$query_identifier` is the unique identifier corresponding to a query (usually this follows a sequential numbering).
   * `Q0` is a legacy field that you can ignore.
   * `$document_identifier` corresponds to the unique identifier of a document (e.g., APXXXXXXX where AP denotes the collection and the Xs correspond to a unique numerical identifier).
   * `$rank_of_document_for_query` denotes the rank of the document for the particular query. This field is ignored by TREC Eval and is only maintained for legacy support. The ranks are computed by TREC Eval itself using the `$query_document_similarity` field (see next). However, it remains good practice to correctly compute this field.
   * `$query_document_similarity` is a score indicating the similarity between query and document where a higher score denotes greater similarity.
   * `$run_identifier` is an identifier of the run. This field is for your own convenience and has no purpose beyond bookkeeping.
   
For example, say we have two queries: `Q1` and `Q2` and we rank three documents (`DOC1`, `DOC2`, `DOC3`). For query `Q1`, we find the following similarity scores `score(Q1, DOC1) = 1.0`, `score(Q1, DOC2) = 0.5`, `score(Q1, DOC3) = 0.75`; and for `Q2`: `score(Q2, DOC1) = -0.1`, `score(Q2, DOC2) = 1.25`, `score(Q1, DOC3) = 0.0`. We can generate run using the following snippet:

In [1]:
# STD
import collections
import io
import logging
from math import log
import sys
import time
import os

# EXT
import numpy as np
import pyndri

# PROJECT
from embeddings import doc_kmeans, doc_centroid, doc_tfidf_scaling, VectorCollection, load_document_representations
from kernels import k_passage, k_gaussian, k_cosine, k_triangle, k_circle
from lsm import LSM
from plm import PLM
from LTR import get_dataset_for_features
from models import LinearRanker, train

# GLOBALS
rankings = collections.defaultdict(lambda: collections.defaultdict(list))


def write_run(model_name, data, out_f,
              max_objects_per_query=sys.maxsize,
              skip_sorting=False):
    """
    Write a run to an output file.
    Parameters:
        - model_name: identifier of run.
        - data: dictionary mapping topic_id to object_assesments;
            object_assesments is an iterable (list or tuple) of
            (relevance, object_id) pairs.
            The object_assesments iterable is sorted by decreasing order.
        - out_f: output file stream.
        - max_objects_per_query: cut-off for number of objects per query.
    """
    for subject_id, object_assesments in data.items():
        if not object_assesments:
            logging.warning('Received empty ranking for %s; ignoring.',
                            subject_id)

            continue

        # Probe types, to make sure everything goes alright.
        # assert isinstance(object_assesments[0][0], float) or \
        #     isinstance(object_assesments[0][0], np.float32)
        assert isinstance(object_assesments[0][1], str) or \
            isinstance(object_assesments[0][1], bytes)

        if not skip_sorting:
            object_assesments = sorted(object_assesments, reverse=True)

        if max_objects_per_query < sys.maxsize:
            object_assesments = object_assesments[:max_objects_per_query]

        if isinstance(subject_id, bytes):
            subject_id = subject_id.decode('utf8')

        for rank, (relevance, object_id) in enumerate(object_assesments):
            if isinstance(object_id, bytes):
                object_id = object_id.decode('utf8')

            out_f.write(
                '{subject} Q0 {object} {rank} {relevance} '
                '{model_name}\n'.format(
                    subject=subject_id,
                    object=object_id,
                    rank=rank + 1,
                    relevance=relevance,
                    model_name=model_name))
            
# The following writes the run to standard output.
# In your code, you should write the runs to local
# storage in order to pass them to trec_eval.
write_run(
    model_name='example',
    data={
        'Q1': ((1.0, 'DOC1'), (0.5, 'DOC2'), (0.75, 'DOC3')),
        'Q2': ((-0.1, 'DOC1'), (1.25, 'DOC2'), (0.0, 'DOC3')),
    },
    out_f=sys.stdout,
    max_objects_per_query=1000)


Q1 Q0 DOC1 1 1.0 example
Q1 Q0 DOC3 2 0.75 example
Q1 Q0 DOC2 3 0.5 example
Q2 Q0 DOC2 1 1.25 example
Q2 Q0 DOC3 2 0.0 example
Q2 Q0 DOC1 3 -0.1 example


Now, imagine that we know that `DOC1` is relevant and `DOC3` is non-relevant for `Q1`. In addition, for `Q2` we only know of the relevance of `DOC3`. The query relevance file looks like:

    Q1 0 DOC1 1
    Q1 0 DOC3 0
    Q2 0 DOC3 1
    
We store the run and qrel in files `example.run` and `example.qrel` respectively on disk. We can now use TREC Eval to compute evaluation measures. In this example, we're only interested in Mean Average Precision and we'll only show this below for brevity. However, TREC Eval outputs much more information such as NDCG, recall, precision, etc.

    $ trec_eval -m all_trec -q example.qrel example.run | grep -E "^map\s"
    > map                   	Q1	1.0000
    > map                   	Q2	0.5000
    > map                   	all	0.7500
    
Now that we've discussed the output format of rankings and how you can compute evaluation measures from these rankings, we'll now proceed with an overview of the indexing framework you'll use.

### Pyndri primer ###
For this assignment you will use [Pyndri](https://github.com/cvangysel/pyndri) [[1](https://arxiv.org/abs/1701.00749)], a python interface for [Indri](https://www.lemurproject.org/indri.php). We have indexed the document collection and you can query the index using Pyndri. We will start by giving you some examples of what Pyndri can do:

First we read the document collection index with Pyndri:

In [2]:
def parse_topics(file_or_files,
                 max_topics=sys.maxsize, delimiter=';'):
    assert max_topics >= 0 or max_topics is None

    topics = collections.OrderedDict()

    if not isinstance(file_or_files, list) and \
            not isinstance(file_or_files, tuple):
        if hasattr(file_or_files, '__iter__'):
            file_or_files = list(file_or_files)
        else:
            file_or_files = [file_or_files]

    for f in file_or_files:
        assert isinstance(f, io.IOBase)

        for line in f:
            assert(isinstance(line, str))

            line = line.strip()

            if not line:
                continue

            topic_id, terms = line.split(delimiter, 1)

            if topic_id in topics and (topics[topic_id] != terms):
                    logging.error('Duplicate topic "%s" (%s vs. %s).',
                                  topic_id,
                                  topics[topic_id],
                                  terms)

            topics[topic_id] = terms

            if max_topics > 0 and len(topics) >= max_topics:
                break

    return topics


# ------------------------
#  Commonly used functions
# ------------------------

def create_index_resources(index_path="index/"):
    index = pyndri.Index(index_path)
    token2id, id2token, id2df = index.get_dictionary()
    dictionary = pyndri.extract_dictionary(index)
    document_ids = list(range(index.document_base(), index.maximum_document()))
    return index, token2id, id2token, id2df, dictionary, document_ids


def create_query_resources(query_path='./ap_88_89/topics_title'):
    with open(query_path, 'r') as f_topics:
        queries = parse_topics([f_topics])

    tokenized_queries = {
        query_id: [dictionary.translate_token(token)
                   for token in index.tokenize(query_string)
                   if dictionary.has_token(token)]
        for query_id, query_string in queries.items()}

    # Record in which query specific query terms are occurring (inverted index)
    query_terms_inverted = collections.defaultdict(set)
    for query_id, query_term_ids in tokenized_queries.items():
        for query_term_id in query_term_ids:
            # A lookup-table for what queries this term appears in
            query_terms_inverted[query_term_id].add(int(query_id))

    query_term_ids = set(
        query_term_id
        for query_term_ids in tokenized_queries.values()
        for query_term_id in query_term_ids)

    return queries, tokenized_queries, query_terms_inverted, query_term_ids


def build_misc_resources(document_ids, query_terms_inverted):
    # Inverted Index creation.
    start_time = time.time()

    inverted_index = collections.defaultdict(lambda: collections.defaultdict(int))
    tf_C = collections.Counter()
    query_word_positions = collections.defaultdict(lambda: collections.defaultdict(list))
    document_lengths = {}
    unique_terms_per_document = {}
    collection_length = 0
    total_terms = 0

    for int_doc_id in document_ids:
        ext_doc_id, doc_token_ids = index.document(int_doc_id)

        for pos, id_at_pos in enumerate(doc_token_ids):
            if id_at_pos in query_term_ids:
                for query_id in query_terms_inverted[id_at_pos]:
                    query_word_positions[int_doc_id][int(query_id)].append(pos)

        document_bow = collections.Counter(
            token_id for token_id in doc_token_ids
            if token_id > 0
        )
        document_length = sum(document_bow.values())

        collection_length += len(doc_token_ids)

        document_lengths[int_doc_id] = document_length
        total_terms += document_length

        unique_terms_per_document[int_doc_id] = len(document_bow)

        for query_term_id in query_term_ids:
            assert query_term_id is not None

            document_term_frequency = document_bow.get(query_term_id, 0)

            if document_term_frequency == 0:
                continue

            inverted_index[query_term_id][int_doc_id] = document_term_frequency
            tf_C[query_term_id] += document_term_frequency

    print('Inverted index creation took', time.time() - start_time, 'seconds.')
    print("Done creating tf_c and query_word_positions.")
    avg_doc_length = total_terms / len(document_ids)
    return inverted_index, tf_C, query_word_positions, unique_terms_per_document, avg_doc_length, document_length, \
           collection_length

The loaded index can be used to access a collection of documents in an easy manner. We'll give you some examples to get some idea of what it can do, it is up to you to figure out how to use it for the remainder of the assignment.

First let's look at the number of documents, since Pyndri indexes the documents using incremental identifiers we can simply take the lowest index and the maximum document and consider the difference:

In [3]:
### We removed the example code to clean up the code

Let's take the first document out of the collection and take a look at it:

Here we see a document consists of two things, a string representing the external document identifier and an integer list representing the identifiers of words that make up the document. Pyndri uses integer representations for words or terms, thus a token_id is an integer that represents a word whereas the token is the actual text of the word/term. Every id has a unique token and vice versa with the exception of stop words: words so common that there are uninformative, all of these receive the zero id.

To see what some ids and their matching tokens we take a look at the dictionary of the index:

Using this dictionary we can see the tokens for the (non-stop) words in our example document:

The reverse can also be done, say we want to look for news about the "University of Massachusetts", the tokens of that query can be converted to ids using the reverse dictionary:

Naturally we can now match the document and query in the id space, let's see how often a word from the query occurs in our example document:

While this is certainly not everything Pyndri can do, it should give you an idea of how to use it. Please take a look at the [examples](https://github.com/cvangysel/pyndri) as it will help you a lot with this assignment.

**CAUTION**: Avoid printing out the whole index in this Notebook as it will generate a lot of output and is likely to corrupt the Notebook.

### Parsing the query file
You can parse the query file (`ap_88_89/topics_title`) using the following snippet:

### Task 1: Implement and compare lexical IR methods [35 points] ### 

In this task you will implement a number of lexical methods for IR using the **Pyndri** framework. Then you will evaluate these methods on the dataset we have provided using **TREC Eval**.

Use the **Pyndri** framework to get statistics of the documents (term frequency, document frequency, collection frequency; **you are not allowed to use the query functionality of Pyndri**) and implement the following scoring methods in **Python**:

- [TF-IDF](http://nlp.stanford.edu/IR-book/html/htmledition/tf-idf-weighting-1.html) and 
- [BM25](http://nlp.stanford.edu/IR-book/html/htmledition/okapi-bm25-a-non-binary-model-1.html) with k1=1.2 and b=0.75. **[5 points]**
- Language models ([survey](https://drive.google.com/file/d/0B-zklbckv9CHc0c3b245UW90NE0/view))
    - Jelinek-Mercer (explore different values of 𝛌 in the range [0.1, 0.5, 0.9]). **[5 points]**
    - Dirichlet Prior (explore different values of 𝛍 [500, 1000, 1500]). **[5 points]**
    - Absolute discounting (explore different values of 𝛅 in the range [0.1, 0.5, 0.9]). **[5 points]**
    - [Positional Language Models](http://sifaka.cs.uiuc.edu/~ylv2/pub/sigir09-plm.pdf) define a language model for each position of a document, and score a document based on the scores of its PLMs. The PLM is estimated based on propagated counts of words within a document through a proximity-based density function, which both captures proximity heuristics and achieves an effect of “soft” passage retrieval. Implement the PLM, all five kernels, but only the Best position strategy to score documents. Use 𝛔 equal to 50, and Dirichlet smoothing with 𝛍 optimized on the validation set (decide how to optimize this value yourself and motivate your decision in the report). **[10 points]**
    
Implement the above methods and report evaluation measures (on the test set) using the hyper parameter values you optimized on the validation set (also report the values of the hyper parameters). Use TREC Eval to obtain the results and report on `NDCG@10`, Mean Average Precision (`MAP@1000`), `Precision@5` and `Recall@1000`.

For the language models, create plots showing `NDCG@10` with varying values of the parameters. You can do this by chaining small scripts using shell scripting (preferred) or execute trec_eval using Python's `subprocess`.

Compute significance of the results using a [two-tailed paired Student t-test](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html) **[5 points]**. Be wary of false rejection of the null hypothesis caused by the [multiple comparisons problem](https://en.wikipedia.org/wiki/Multiple_comparisons_problem). There are multiple ways to mitigate this problem and it is up to you to choose one.

Analyse the results by identifying specific queries where different methods succeed or fail and discuss possible reasons that cause these differences. This is *very important* in order to understand who the different retrieval functions behave.

**NOTE**: Don’t forget to use log computations in your calculations to avoid underflows. 

**IMPORTANT**: You should structure your code around the helper functions we provide below.

In [4]:
# ------------------------------------------------
# Task 1: Implement and compare lexical IR methods
# ------------------------------------------------


def run_retrieval(index, model_name, queries, document_ids, scoring_func, max_objects_per_query=1000,
                  target_dir="./lexical_results/", **resource_params):
    """
    Runs a retrieval method for all the queries and writes the TREC-friendly results in a file.

    :param index: Pyndri index object.
    :type index: pyndri.Index
    :param model_name: Name of the model (used for screen printing).
    :type model_name: str
    :param queries: Queries to be evaluated.
    :type queries: dict
    :param document_ids: Collection of document ids in data set.
    :type document_ids: list or set or tuple
    :param scoring_func: Function that uses the prepared resources, query and document id to score a query and a doc.
    :type scoring_func: func
    :param max_objects_per_query: Only keep a certain number of best ranked documents per query.
    :type max_objects_per_query: int
    :param resource_params: Named arguments that are used to build resources used by the model for scoring.
    :type resource_params: dict
    """
    # The dictionary data should have the form: model_name --> query_id --> (document_score, external_doc_id)
    global rankings, reranking_ids
    run_out_path = '{}.run'.format(model_name)

    run_id = 0
    while os.path.exists(run_out_path):
        run_id += 1
        run_out_path = '{}_{}.run'.format(model_name, run_id)

    print('Retrieving using', model_name)
    start = time.time()
    query_times = 0
    n_queries = len(queries)

    for i, query in enumerate(queries.items()):
        query_start_time = time.time()
        query_id, _ = query
        query_scores = []

        # Do actual scoring here
        for n, document_id in enumerate(document_ids):
            ext_doc_id, document_word_positions = index.document(document_id)
            score = scoring_func(index, query_id, document_id, **resource_params)
            query_scores.append((score, ext_doc_id))

        query_id = int(query_id)
        rankings[model_name][query_id] = list(sorted(query_scores, reverse=True))[:max_objects_per_query]

        query_end_time = time.time()
        query_time = query_end_time - query_start_time
        query_times += query_time
        average_query_time = query_times / max(i, 1)
        querys_left = len(queries) - i
        time_left = average_query_time * querys_left
        m, s = divmod(time_left, 60)
        h, m = divmod(m, 60)
        print(
            "\rAverage query time for query {} out of {}: {:.2f} seconds. {} hour(s), {} minute(s) and {:.2f} seconds "
            "remaining for {} queries.".format(
                i+1, n_queries, average_query_time, int(h), int(m), s, querys_left
            ), flush=True, end=""
        )

    # Write results
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)

    with open('{}{}'.format(target_dir, run_out_path), 'w') as f_out:
        write_run(
            model_name=model_name,
            data=rankings[model_name],
            out_f=f_out,
            max_objects_per_query=max_objects_per_query)

    print("Done writing results to run file")

    end = time.time()
    print("Retrieval took {:.2f} seconds.".format(end-start))


def idf(term_id, id2df, num_documents):
    df_t = id2df[term_id]
    return log(num_documents) - log(df_t)


def tf_idf(index, query_id, document_id, document_term_freqs, tokenized_queries, id2df, num_documents,
           **resource_params):
    log_sum = 0

    for query_term_id in tokenized_queries[query_id]:
        # TODO: How are we adding the values here?
        log_sum += log(1 + document_term_freqs[query_term_id][document_id]) * idf(query_term_id, id2df, num_documents)

    return log_sum


def bm25(index, query_id, document_id, document_term_freqs, avg_doc_length, id2df, num_documents, tokenized_queries,
         **resource_params):
    """
    :param document_id:
    :param term_id:
    :param document_term_freq: How many times this term appears in the given document
    :_ unused tuning parameter
    :returns: A score for the given document in the light of the given query term

    Since all the scoring functions are supposed to only score one query term,
    the BM25 summation is being done outside this function.
    Do note that we leave the k_3 factor out, since all the queries
    in this assignment can be assumed to be reasonably short.
    """

    def bm25_formula(document_id, query_term_id, document_term_freqs, l_d, l_average, id2df, num_documents):
        enumerator = (k_1 + 1) * document_term_freqs[query_term_id][document_id]
        denominator = k_1 * ((1-b) + b * (l_d/l_average)) + document_term_freqs[query_term_id][document_id]
        bm25_score = idf(query_term_id, id2df, num_documents) * enumerator/denominator

        if bm25_score == 0:
            return 0
        return bm25_score

    k_1 = 1.2
    b = 0.75
    l_average = avg_doc_length
    l_d = index.document_length(document_id)

    sum_ = 0
    for query_term_id in tokenized_queries[query_id]:
        # TODO: How do we combine the values here?
        sum_ += bm25_formula(document_id, query_term_id, document_term_freqs, l_d, l_average, id2df, num_documents)

    return sum_


def LM_jelinek_mercer_smoothing(index, query_id, document_id, document_term_freqs, collection_length, tf_C,
                                tokenized_queries, tuning_parameter=0.1, **resource_params):
    log_sum = 0

    for query_term_id in tokenized_queries[query_id]:
        tf = document_term_freqs[query_term_id][document_id]
        lamb = tuning_parameter
        doc_length = index.document_length(document_id)
        C = collection_length

        try:
            prob_q_d = lamb * (tf / doc_length) + (1 - lamb) * (tf_C[query_term_id] / C)
        except ZeroDivisionError:
            prob_q_d = 0

        log_sum += np.log(prob_q_d)

    return log_sum


def LM_dirichlet_smoothing(index, query_id, document_id, document_term_freqs, collection_length, tokenized_queries,
                           tuning_parameter=500, **resource_params):
    log_sum = 0

    for query_term_id in tokenized_queries[query_id]:
        tf = document_term_freqs[query_term_id][document_id]
        mu = tuning_parameter
        doc_length = index.document_length(document_id)
        C = collection_length

        prob_q_d = (tf + mu * (tf_C[query_term_id] / C)) / (doc_length + mu)

        log_sum += np.log(prob_q_d)

    return log_sum

def LM_absolute_discounting(index, query_id, document_id, document_term_freq, num_unique_words, collection_length,
                         tokenized_queries, tuning_parameter=0.1):
    log_sum = 0

    for query_term_id in tokenized_queries[query_id]:
        discount = tuning_parameter
        d = index.document_length(document_id)
        C = collection_length
        if d == 0: return -9999
        number_of_unique_terms = num_unique_words[document_id]

        log_sum += np.log(
            max(document_term_freq - discount, 0) / d + ((discount * number_of_unique_terms) / d) *
            (tf_C[query_term_id] / C)
        )

    return log_sum


def create_all_lexical_run_files(index, document_ids, queries, document_term_freqs, collection_length, tf_C,
                                 tokenized_queries, background_model, idf2df, num_documents):
    print("##### Creating all lexical run files! #####")
    start = time.time()
    print("Running TFIDF")
    run_retrieval(
        index, 'tfidf', queries, document_ids, tf_idf,
        document_term_freqs=document_term_freqs, tokenized_queries=tokenized_queries, id2df=idf2df,
        num_documents=num_documents
    )
    end = time.time()
    print("Retrieval took {:.2f} seconds.".format(end-start))

    start = time.time()
    print("Running BM25")
    run_retrieval(
        index, 'bm25', queries, document_ids, bm25,
        document_term_freqs=document_term_freqs, avg_doc_length=avg_doc_length, id2df=id2df,
        num_documents=num_documents, tokenized_queries=tokenized_queries
    )
    end = time.time()
    print("Retrieval took {:.2f} seconds.".format(end-start))

    j_m__lambda_values = [0.1, 0.3, 0.5, 0.7, 0.9]
    for val in j_m__lambda_values:
        start = time.time()
        print("Running LM_jelinek", val)
        run_retrieval(
            index, 'LM_jelinek_mercer_smoothing_{}'.format(str(val).replace(".", "_")),
            queries, document_ids, LM_jelinek_mercer_smoothing,
            tuning_parameter=val, document_term_freqs=document_term_freqs, collection_length=collection_length,
            tf_C=tf_C, tokenized_queries=tokenized_queries
        )
        end = time.time()
        print("Retrieval took {:.2f} seconds.".format(end-start))

    dirichlet_values = [500, 1000, 1500]
    for val in dirichlet_values:
        start = time.time()
        print("Running Dirichlet", val)
        run_retrieval(
            index, 'LM_dirichelt_smoothing_{}'.format(str(val).replace(".", "_")),
            document_ids, queries, LM_dirichlet_smoothing,
            tuning_parameter=val, document_term_freqs=document_term_freqs, collection_length=collection_length,
            tokenized_queries=tokenized_queries
        )
        end = time.time()
        print("Retrieval took {:.2f} seconds.".format(end-start))

    absolute_discounting_values = j_m__lambda_values
    for val in absolute_discounting_values:
        start = time.time()
        print("Running ABS_discount", val)
        run_retrieval('LM_absolute_discounting_{}'.format(str(val).replace(".", "_")), LM_absolute_discounting, document_ids, tuning_parameter=val)
        end = time.time()
        print("Retrieval took {:.2f} seconds.".format(end-start))

    start = time.time()
    run_retrieval_plm(
        index, 'PLM_passage', queries, document_ids, query_word_positions, background_model, tokenized_queries,
        collection_length, kernel=k_passage
    )

    start = time.time()
    run_retrieval_plm(
        index, 'PLM_gaussian', queries, document_ids, query_word_positions, background_model, tokenized_queries,
        collection_length, kernel=k_gaussian
    )
    end = time.time()
    print("Retrieval took {:.2f} seconds.".format(end-start))

    start = time.time()
    run_retrieval_plm(
        index, 'PLM_triangle', queries, document_ids, query_word_positions, background_model, tokenized_queries,
        collection_length, kernel=k_triangle
    )
    end = time.time()
    print("Retrieval took {:.2f} seconds.".format(end-start))

    start = time.time()
    run_retrieval_plm(
        index, 'PLM_cosine', queries, document_ids, query_word_positions, background_model, tokenized_queries,
        collection_length, kernel=k_cosine
    )
    end = time.time()
    print("Retrieval took {:.2f} seconds.".format(end-start))

    start = time.time()
    run_retrieval_plm(
        index, 'PLM_circle', queries, document_ids, query_word_positions, background_model, tokenized_queries,
        collection_length, kernel=k_circle
    )
    end = time.time()
    print("Retrieval took {:.2f} seconds.".format(end-start))

### Task 2: Latent Semantic Models (LSMs) [15 points] ###

In this task you will experiment with applying distributional semantics methods ([LSI](http://lsa3.colorado.edu/papers/JASIS.lsi.90.pdf) **[5 points]** and [LDA](https://www.cs.princeton.edu/~blei/papers/BleiNgJordan2003.pdf) **[5 points]**) for retrieval.

You do not need to implement LSI or LDA on your own. Instead, you can use [gensim](http://radimrehurek.com/gensim/index.html). An example on how to integrate Pyndri with Gensim for word2vec can be found [here](https://github.com/cvangysel/pyndri/blob/master/examples/word2vec.py). For the remaining latent vector space models, you will need to implement connector classes (such as `IndriSentences`) by yourself.

In order to use a latent semantic model for retrieval, you need to:
   * build a representation of the query **q**,
   * build a representation of the document **d**,
   * calculate the similarity between **q** and **d** (e.g., cosine similarity, KL-divergence).
     
The exact implementation here depends on the latent semantic model you are using. 
   
Each of these LSMs come with various hyperparameters to tune. Make a choice on the parameters, and explicitly mention the reasons that led you to these decisions. You can use the validation set to optimize hyper parameters you see fit; motivate your decisions. In addition, mention clearly how the query/document representations were constructed for each LSM and explain your choices.

In this experiment, you will first obtain an initial top-1000 ranking for each query using TF-IDF in **Task 1**, and then re-rank the documents using the LSMs. Use TREC Eval to obtain the results and report on `NDCG@10`, Mean Average Precision (`MAP@1000`), `Precision@5` and `Recall@1000`.

Perform significance testing **[5 points]** (similar as in Task 1) in the class of semantic matching methods.

In [5]:
# ------------------------------
# Task 2: Latent Semantic Models
# ------------------------------

def lsm_reranking(ranked_queries, LSM_model):
    reranking = collections.defaultdict(list)
    LSM_model.similarity_index.num_best = None

    for query_id, docs in ranked_queries.items():
        query_terms = [id2token[term_id] for term_id in tokenized_queries[query_id]]
        query_bow = LSM_model.dictionary.doc2bow(query_terms)
        query_lsm = LSM_model.model[query_bow]

        # Get similarity of query with all documents
        sims = LSM_model.similarity_index[query_lsm]

        for document_id in document_ids:
            ext_doc_id, _ = index.document(document_id)
            reranking[query_id].append((sims[document_id-1], ext_doc_id))

    for query_id in reranking:
        reranking[query_id] = list(sorted(reranking[query_id], reverse=True))[:1000]

    return reranking


def create_latent_semantic_model_runfiles():
    global rankings

    # LSI
    start = time.time()
    lsi = LSM('LSI', index)
    lsi.create_model()
    end = time.time()
    print("LSI model creation took {:.2f} seconds.".format(end - start))

    start = time.time()
    lsi.create_similarity_index()
    end = time.time()
    print("LSI similarity index creation took {:.2f} seconds.".format(end - start))

    start = time.time()
    lsi_reranking = lsm_reranking(ranked_queries=rankings['tfidf'], LSM_model=lsi)
    end = time.time()
    print("LSI reranking took {:.2f} seconds.".format(end - start))

    start = time.time()
    run_out_path = '{}.run'.format('LSI')
    with open('./lexical_results/{}'.format(run_out_path), 'w') as f_out:
        write_run(
            model_name='LSI',
            data=lsi_reranking,
            out_f=f_out,
            max_objects_per_query=1000)
    end = time.time()
    print("LSI run file creation {:.2f} seconds.".format(end - start))

    # LDA
    start = time.time()
    lda = LSM('LDA', index)
    lda.create_model()
    end = time.time()
    print("LDA model creation took {:.2f} seconds.".format(end - start))

    start = time.time()
    lda.create_similarity_index()
    end = time.time()
    print("LDA similarity index creation took {:.2f} seconds.".format(end - start))

    start = time.time()
    lda_reranking = lsm_reranking(ranked_queries=rankings['tfidf'], LSM_model=lda)
    end = time.time()
    print("LDA reranking took {:.2f} seconds.".format(end - start))

    start = time.time()
    run_out_path = '{}.run'.format('LDA')
    with open('./lexical_results/{}'.format(run_out_path), 'w') as f_out:
        write_run(
            model_name='LDA',
            data=lda_reranking,
            out_f=f_out,
            max_objects_per_query=1000)
    end = time.time()
    print("LDA run file creation {:.2f} seconds.".format(end - start))

### Task 3:  Word embeddings for ranking [20 points] (open-ended) ###

First create word embeddings on the corpus we provided using [word2vec](http://arxiv.org/abs/1411.2738) -- [gensim implementation](https://radimrehurek.com/gensim/models/word2vec.html). You should extract the indexed documents using pyndri and provide them to gensim for training a model (see example [here](https://github.com/nickvosk/pyndri/blob/master/examples/word2vec.py)).
   
This is an open-ended task. It is left up you to decide how you will combine word embeddings to derive query and document representations. Note that since we provide the implementation for training word2vec, you will be graded based on your creativity on combining word embeddings for building query and document representations.

Note: If you want to experiment with pre-trained word embeddings on a different corpus, you can use the word embeddings we provide alongside the assignment (./data/reduced_vectors_google.txt.tar.gz). These are the [google word2vec word embeddings](https://code.google.com/archive/p/word2vec/), reduced to only the words that appear in the document collection we use in this assignment.

In [6]:
# -----------------------------------
# Task 3 Word Embeddings for Ranking
# -----------------------------------
import h5py
import numpy as np
from gensim.models import Word2Vec
from sklearn.cluster import KMeans

def doc_centroid(vectors, cache=None, **kwargs):
    """ Calculate the centroid of some vectors. """
    return np.add.reduce(vectors) / len(vectors) if len(vectors) > 0 else np.array([]), cache


def _doc_kmeans(vectors, k=None, **kwargs):
    """
    Helper function for doc_kmeans.
    Calculate the centroid of vectors after clustering them using k-means and only keeping those which belong to the
    biggest cluster. If k is None, it will be chosen using a heuristic.
    """
    # Simple cases
    if len(vectors) == 0:
        return np.array([])
    if len(vectors) == 1:
        return vectors

    def most_common(lst):
        data = Counter(lst)
        return data.most_common(1)[0][0]

    # Dirty but fast heuristic
    if k is None:
        k = int(np.sqrt(len(vectors) / 2))

    kmeans = KMeans(n_clusters=k).fit(vectors)
    labels = kmeans.labels_
    biggest_cluster = most_common(labels)  # Get label of biggest cluster

    # Return the vectors belonging to it
    return [vector for vector, label in zip(vectors, labels) if label == biggest_cluster]


def doc_kmeans(vectors, k=None, cache=None, **kwargs):
    """
    Calculate the centroid of vectors after clustering them using k-means and only keeping those which belong to the
    biggest cluster. If k is None, it will be chosen using a heuristic.
    """
    filtered_vectors = _doc_kmeans(vectors, k)

    # Cache most recent vectors for doc_kmeans_tfidf so you don't have to do it twice
    if cache is not None:
        cache["kmeans_filtered"] = filtered_vectors
    return doc_centroid(filtered_vectors), cache  # Calculate the centroid with vector from biggest cluster.


def doc_tfidf_scaling(vectors, token_ids, document_term_freqs, id2df, number_of_documents, document_id, cache=dict(),
                      **kwargs):
    """
    Calculate the centroid of a cluster, but scale the vectors by their tf-idf value.
    """
    # Easy case, empty document
    if len(vectors) == 0:
        return np.array([])

    for i, (vector, token_id) in enumerate(zip(vectors, token_ids)):
        # Use caching
        if hasattr(cache, "tfidf"):
            tf_idf_values = cache["tfidf"]
        else:
            tf_idf_values = cache["tfidf"] = dict()
        term_tf_idf = tf_idf_values.get(
            (token_id, document_id),
            tf_idf(token_id, document_term_freqs[token_id][document_id], id2df, number_of_documents)
        )
        cache["tfidf"][(token_id, document_id)] = term_tf_idf

        vectors[i] = vector * term_tf_idf  # Scale

    return doc_centroid(vectors), cache  # Now compute centroid


class VectorCollection:
    """ Class to store word embeddings created by Word2Vec. """
    def __init__(self, word_vectors, context_vectors):
        self.word_vectors = {
            word: word_vectors.syn0[vocab_item.index] for word, vocab_item in word_vectors.vocab.items()
        }  # W_in vectors
        self.context_vectors = {
            word_vectors.index2word[i]: vec for i, vec in enumerate(context_vectors)
        }  # W_out vectors

    @staticmethod
    def load_vectors(path, to_train=False):
        """
        Load word embeddings created by gensim Word2Vec from a directory.
        If the to_train flag is set, the whole model will be returned, otherwise just an instance of VectorCollection
        only containing the embeddings to save memory.
        """
        model = Word2Vec.load(path)

        if to_train:
            return model

        # In case it doesn't need to be trained, delete train code to free up ram
        word_vectors = model.wv

        context_vectors = dict()
        if hasattr(model, "syn1"):
            # For hierarchical softmax
            context_vectors = model.syn1
        elif hasattr(model, "syn1neg"):
            # For negative sampling
            context_vectors = model.syn1neg

        del model  # Save memory
        return VectorCollection(word_vectors, context_vectors)


def calculate_document_representations(pyndri_index, vector_collection, document_ids, *combination_funcs,
                                       vector_func=lambda word, collection: collection.word_vectors[word], **kwargs):
    """
    Pre-compute document vector representations to speed up retrieval later.
    :param: pyndri_index: Pyndri index object
    :type: pyndri_index: pyndri.index
    :param: vector_collection: VectorCollection containing both word and context vectors.
    :type: VectorCollection
    :param: document_ids: IDs of all the documents inside the index.
    :type: set or list or tuple
    :param: combination_funcs: Functions that are used to combine word embeddings.
    :type: tuple of functions
    :param: vector_func: Function used to retrieve an embedding from the vector collection (is useful to determine
    whether word or context vectors are being used.)
    :type: vector_func: func
    """
    representations = {func.__name__: dict() for func in combination_funcs}  # Create result dict

    token2id, id2token, _ = pyndri_index.get_dictionary()
    unkowns = set()  # Set of unknown words for which no embedding exists
    n_documents = len(document_ids)
    cache = dict()  # For some calculations, caching some computations is helpful

    for i, document_id in enumerate(document_ids):
        print("\r{:.2f} % of documents processed.".format((i+1)/n_documents*100), end="", flush=True)

        _, token_ids = pyndri_index.document(document_id)

        token_ids = [token_id for token_id in token_ids if token_id != 0]  # Filter out stop words
        vectors = []
        for token_id in token_ids:
            word = id2token[token_id]
            try:
                vectors.append(vector_func(word, vector_collection))
            except KeyError:
                unkowns.add(word)

        for func in combination_funcs:
            # Create the representations, store them, discard them if the document was empty.
            (representation, _), cache = func(
                vectors, token_ids=token_ids, document_id=document_id, cache=cache, **kwargs
            )
            if representation.shape == (0, ): break
            representations[func.__name__][document_id] = representation

    print("\n{} unknown words encountered.".format(len(unkowns)))

    return representations


def save_document_representations(reps, path):
    """ Store word embeddings using the HDF5 file format. """
    file = h5py.File(path, 'w')

    for name, vector_dict in reps.items():
        vectors = [vector_dict[i] for i in vector_dict.keys()]
        file.create_dataset(name, data=vectors)

    file.close()


def load_document_representations(path):
    """ Load word embeddings using the HDF5 file format. """
    return h5py.File(path, "r")


def run_retrieval_embeddings_So(index, model_name, queries, document_ids, id2token, vector_collection,
                                document_representations, tokenized_queries, combination_func, doc2repr=None,
                                **resource_params):
    """
    Score a document and a query using the cosine similarity between the document and the query vector
    representation. 
    """
    def score_func(index, query_id, document_id, **resource_params):
        id2token = resource_params["id2token"]
        vector_collection = resource_params["vector_collection"]
        tokenized_queries = resource_params["tokenized_queries"]
        query_token_ids = tokenized_queries[query_id]
        doc2repr = resource_params.get("doc2repr", None)
        vector_func_query = resource_params.get(
            "vector_func_query", lambda word, collection: collection.word_vectors[word]
        )
        query_vectors = []
        for query_token_id in query_token_ids:
            if query_token_id == 0: continue
            try:
                query_vectors.append(vector_func_query(id2token[query_token_id], vector_collection))
            except KeyError:
                # OOV word
                continue
        if len(query_vectors) == 0:
            return -1

        query_vector = combination_func(query_vectors, document_id=document_id,
                                        token_ids=tokenized_queries[query_id], **resource_params)
        query_vector = unpack_vec(query_vector)

        try:
            lookup_id = document_id if doc2repr is None else doc2repr[document_id]
            document_vector = document_representations[lookup_id]
            document_vector = unpack_vec(document_vector)
        except KeyError as ie:
            # Empty documents, give worst score
            return -1

        return cosine_similarity(query_vector, document_vector)

    return run_retrieval(
        index, model_name, queries, document_ids, score_func,
        # Named key word arguments to build as resources before scoring
        id2token=id2token, vector_collection=vector_collection, document_representations=document_representations,
        combination_func=combination_func, doc2repr=doc2repr, tokenized_queries=tokenized_queries, **resource_params
    )


def run_retrieval_embeddings_Savg(index, model_name, queries, document_ids, id2token, vector_collection,
                             document_representations, tokenized_queries, doc2repr=None):
    """
    Score a document and a query using the average cosine similarity between the document and the all
    of the query word embeddings.
    """
    def score_func(index, query_id, document_id, **resource_params):
        id2token = resource_params["id2token"]
        vector_collection = resource_params["vector_collection"]
        tokenized_queries = resource_params["tokenized_queries"]
        doc2repr = resource_params.get("doc2repr", None)
        query_token_ids = tokenized_queries[query_id]
        vector_func_query = resource_params.get(
            "vector_func_query", lambda word, collection: collection.word_vectors[word]
        )
        query_vectors = []
        for query_token_id in query_token_ids:
            if query_token_id == 0: continue
            try:
                query_vectors.append(vector_func_query(id2token[query_token_id], vector_collection))
            except KeyError:
                # OOV word
                continue
        if len(query_vectors) == 0:
            return -1

        try:
            lookup_id = document_id if doc2repr is None else doc2repr[document_id]
            document_vector = document_representations[lookup_id]
        except KeyError:
            # Empty documents, give worst score
            return -1

        return sum(
            [cosine_similarity(query_vector, document_vector) for query_vector in query_vectors]
        ) / len(query_vectors)

    return run_retrieval(
        index, model_name, queries, document_ids, score_func,
        # Named key word arguments to build as resources before scoring
        id2token=id2token, vector_collection=vector_collection, document_representations=document_representations,
        tokenized_queries=tokenized_queries, doc2repr=doc2repr
    )


def run_retrieval_plm(index, model_name, queries, document_ids, query_word_positions, background_model,
                      tokenized_queries, collection_length, kernel=k_gaussian):
    def score_func(index, query_id, document_id, **resource_params):
        query_word_positions = resource_params["query_word_positions"]
        background_model = resource_params.get("background_model", None)
        document_length = index.document_length(document_id)
        query_term_positions_for_document = query_word_positions[document_id][int(query_id)]

        # If none of the query terms appear in this document, the score is 0
        if not query_term_positions_for_document:
            return 0

        query_term_ids = tokenized_queries[query_id]

        plm = PLM(
            query_term_ids, document_length, query_term_positions_for_document,
            background_model=background_model, kernel=kernel, collection_length=collection_length
        )
        score = plm.best_position_strategy_score()
        print("Score", score)
        return score

    return run_retrieval(
        index, model_name, queries, document_ids, score_func,
        # Named key word arguments to build as resources before scoring
        query_word_positions=query_word_positions, kernel=kernel, background_model=background_model,
        tokenized_queries=tokenized_queries
    )


def cosine_similarity(a, b):
    """Takes 2 vectors a, b and returns the cosine similarity according
    to the definition of the dot product
    """
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    res = dot_product / (norm_a * norm_b)
    if np.isnan(res):
        # If one of the vectors have zero length,
        # we can not score the similarity between the two vectors, so we assume the worst
        return -1

    return res


def create_document_id_to_repr_map(document_ids):
    """
    Compensate for empty documents in document collections.
    Because of using hd5 storage, we can only store vectors of the same length, so no zero-length vectors for empty
    documents. Because they were disregarded during the strorage procedure, the alignment between document ids and
    indices of the list with their vector representations is now off and has to be fixed.

    It's a little dirty and hack-y, but was the best solution given the time constraint.
    """
    empty_ids = {93688, 102435, 104040, 121863, 121866, 122113, 147904, 149905, 153512, 154467, 155654}
    doc2repr = dict()

    zone = 1
    for i in range(1, len(document_ids)+1):
        if i in empty_ids:
            zone += 1
            continue

        doc2repr[i] = i - zone

    return doc2repr


def unpack_vec(vector):
    while type(vector) == tuple:
        vector = vector[0]

    return vector


def eval_word_embedding_models(index, queries, document_ids, tf_idf, tokenized_queries, id2df, num_documents,
                               document_term_freqs, inverted_index):

    # Use tf-idf for re-ranking

    run_retrieval(
        index, 'tfidf', queries, document_ids, tf_idf,
        document_term_freqs=document_term_freqs, tokenized_queries=tokenized_queries, id2df=id2df,
        num_documents=num_documents
    )
    print("Reading word embeddings...")
    vectors = VectorCollection.load_vectors("./w2v_60")
    doc2repr = create_document_id_to_repr_map(document_ids)
    print("Reading document representations...")
    doc_representations = load_document_representations("./win_representations_1_4")
    # OUT - IN
    run_retrieval_embeddings_So(
        index, "embeddings_So_centroid_wout_win", queries, document_ids, id2token=id2token, vector_collection=vectors,
        document_representations=doc_representations.get("doc_centroid"), tokenized_queries=tokenized_queries,
        doc2repr=doc2repr, combination_func=doc_centroid,
        vector_func_query=lambda word, collection: collection.context_vectors[word],
        document_term_freqs=inverted_index, id2df=id2df, number_of_documents=num_documents, cache=dict()
    )
    del doc_representations
    doc_representations = load_document_representations("./wout_representations_1_4")

    # IN - OUT
    run_retrieval_embeddings_So(
        index, "embeddings_So_centroid_win_wout", queries, document_ids, id2token=id2token, vector_collection=vectors,
        document_representations=doc_representations.get("doc_centroid"), tokenized_queries=tokenized_queries,
        doc2repr=doc2repr, combination_func=doc_centroid,
        document_term_freqs=inverted_index, id2df=id2df, number_of_documents=num_documents, cache=dict()
    )

    # OUT - OUT
    run_retrieval_embeddings_So(
        index, "embeddings_So_centroid_wout_wout", queries, document_ids, id2token=id2token, vector_collection=vectors,
        document_representations=doc_representations.get("doc_centroid"), tokenized_queries=tokenized_queries,
        doc2repr=doc2repr, combination_func=doc_centroid,
        vector_func_query=lambda word, collection: collection.context_vectors[word],
        document_term_freqs=inverted_index, id2df=id2df, number_of_documents=num_documents, cache=dict()
    )


ImportError: No module named 'h5py'

### Task 4: Learning to rank (LTR) [15 points] (open-ended) ###

In this task you will get an introduction into learning to rank for information retrieval.

You can explore different ways for devising features for the model. Obviously, you can use the retrieval methods you implemented in Task 1, Task 2 and Task 3 as features. Think about other features you can use (e.g. query/document length). Creativity on devising new features and providing motivation for them will be taken into account when grading.

For every query, first create a document candidate set using the top-1000 documents using TF-IDF, and subsequently compute features given a query and a document. Note that the feature values of different retrieval methods are likely to be distributed differently.

You are adviced to start some pointwise learning to rank algorithm e.g. logistic regression, implemented in [scikit-learn](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html).
Train your LTR model using 10-fold cross validation on the test set. More advanced learning to rank algorithms will be appreciated when grading.

In [ ]:
# ------------------------------
# Task 4: Learning to rank (LTR)
# ------------------------------
def get_document_id_maps(index, document_ids):
    id2ext = {}
    ext2id = {}

    for document_id in document_ids:
        ext_doc_id, doc_token_ids = index.document(document_id)
        id2ext[document_id] = ext_doc_id
        ext2id[ext_doc_id] = document_id

    return id2ext, ext2id


def get_top_tf_idf_documents(queries, document_ids, index, max_objects_per_query=1000):

    top_documents_for_query = collections.defaultdict(list)

    id2ext, ext2id = get_document_id_maps(index, document_ids)

    with open("./lexical_results/tfidf.run", "r") as f:
        for line in f.readlines():
            query_id, _, external_document_id, ranking, score, class_name = line.split()
            top_documents_for_query[query_id].append(\
                (float(score), ext2id[external_document_id], external_document_id))


    return top_documents_for_query

def extract_values_from_run_file(filepath):
    # Since we have already calculated a lot of the feature values in previous runs
    # we can use these values and only calculate the remaining ones
    cache = collections.defaultdict(lambda: collections.defaultdict(float))
    with open(filepath, "r") as f:
        for line in f.readlines():
            query_id, _, document_id, ranking, score, class_name = line.split()
            # Append feature values to the features lookup table directly
            cache[int(query_id)][document_id] = float(score)

    return cache

def write_features_to_file(features, filepath):
    s = ""

    for query_id, documents in features.items():
        for document_id, feature_vector in documents.items():
            s += "{} {} {}\n".format(query_id, document_id, [feature for feature in feature_vector])

    with open(filepath, "w") as f:
        f.write(s)


def extract_features(queries, document_ids, index,\
            document_term_freqs, avg_doc_length, id2df, num_documents, tokenized_queries, collection_length):
    """
    Goal: return features[query_id][document_id] = feature_vector

    """
    features = collections.defaultdict(lambda: collections.defaultdict(list))

    documents_for_query = get_top_tf_idf_documents(queries, document_ids, index)


    bm25_cache = extract_values_from_run_file("./lexical_results/bm25.run")
    jm_cache = extract_values_from_run_file("./lexical_results/LM_jelinek_mercer_smoothing_0_7.run")

    i = -1
    for query_id, documents in documents_for_query.items():
        i += 1
        print("Extracting features for query nr {}".format(i))
        for tf_idf_score, document_id, external_document_id in documents:
            ### The different features

            # tf-idf
            features[int(query_id)][external_document_id].append(float(tf_idf_score))

            # bm25
            if external_document_id in bm25_cache[query_id]:
                score = bm25_cache[query_id][external_document_id]

            else:
                score = bm25(index, query_id, document_id, document_term_freqs, \
                        avg_doc_length, id2df, num_documents, tokenized_queries)

            features[int(query_id)][external_document_id].append(float(score))

            # JM
            if external_document_id in jm_cache[query_id]:
                score = jm_cache[query_id][external_document_id]

            else:
                score = LM_jelinek_mercer_smoothing(index, query_id, document_id, document_term_freqs, collection_length, tf_C,
                                tokenized_queries)
            features[int(query_id)][external_document_id].append(float(score))

            # Document length
            features[int(query_id)][external_document_id].append(index.document_length(document_id))

            # Query length
            features[int(query_id)][external_document_id].append(len(tokenized_queries[query_id]))

    write_features_to_file(features, "./features.txt")
    return features

def evaluate_model(model, test_features, run_out_path, cross_id):
    scores = collections.defaultdict(list)

    for feature in test_features:
        query_id, ext_document_id, feature_vector = feature
        feature_vector = np.array(feature_vector)
        score = model.predict(feature_vector.reshape(1, -1))
        scores[int(query_id)].append((float(score), ext_document_id))

    with open('{}{}.run'.format(run_out_path, cross_id), 'w') as f_out:
        write_run(
            model_name="regression",
            data=scores,
            out_f=f_out,
            max_objects_per_query=1000)

if __name__ == "__main__":
    index, token2id, id2token, id2df, dictionary, document_ids = create_index_resources()
    num_documents = len(document_ids)

    queries, tokenized_queries, query_terms_inverted, query_term_ids = create_query_resources()

    inverted_index, tf_C, query_word_positions, unique_terms_per_document, avg_doc_length, document_length, \
        collection_length = build_misc_resources(document_ids, query_terms_inverted)
    document_term_freqs = inverted_index

    create_all_lexical_run_files(
        index, document_ids, queries, document_term_freqs, collection_length, tf_C,
        tokenized_queries, background_model=tf_C, idf2df=id2df, num_documents=num_documents
    )
    features = extract_features(queries, document_ids, index,\
            document_term_freqs, avg_doc_length, id2df, num_documents, tokenized_queries, collection_length)

    datasets, test_features = zip(*[get_dataset_for_features(features, i=i) for i in range(10)])
    test_features = test_features[0]

    id2ext, ext2id = get_document_id_maps(index, document_ids)

    for i, dataset in enumerate(datasets):
        model = train(dataset)
        evaluate_model(model, test_features, "./regression", i)


### Task 4: Write a report [15 points; instant FAIL if not provided] ###

The report should be a PDF file created using the [sigconf ACM template](https://www.acm.org/publications/proceedings-template) and will determine a significant part of your grade.

   * It should explain what you have implemented, motivate your experiments and detail what you expect to learn from them. **[10 points]**
   * Lastly, provide a convincing analysis of your results and conclude the report accordingly. **[10 points]**
      * Do all methods perform similarly on all queries? Why?
      * Is there a single retrieval model that outperforms all other retrieval models (i.e., silver bullet)?
      * ...

**Hand in the report and your self-contained implementation source files.** Only send us the files that matter, organized in a well-documented zip/tgz file with clear instructions on how to reproduce your results. That is, we want to be able to regenerate all your results with minimal effort. You can assume that the index and ground-truth information is present in the same file structure as the one we have provided.
